In [1]:
import numpy as np

In [2]:
hf = {
    'g1':{
        'signals': np.random.rand(20, 8, 8),
        'labels': np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])
        },
    'g2':{
        'signals': np.random.rand(24, 8, 8),
        'labels': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    },
    'g3':{
        'signals': np.random.rand(26, 8, 8),
        'labels': np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
    }
}
elm_indices = ['g1', 'g2', 'g3']

In [3]:
signals = None
signal_window_size = 8
for it, elm_idx in enumerate(elm_indices):
    print(f'-----Iteration: {it}-----')
    elm_event = hf[elm_idx]
    signals_np = elm_event['signals']
    labels_np = elm_event['labels']
    valid_t0_np = np.ones(labels_np.shape)
    valid_t0_np[-signal_window_size+1:] = 0
    print(f'valid_t0_np: {valid_t0_np}')
    active_elm_indices = np.where(labels_np > 0)[0]
    print(f'Active ELM indices: {active_elm_indices}')
    if signals is None:
        window_start_indices = np.array([0], 'int')
        print(f'window start indices: {window_start_indices}')
        elm_start_indices = np.array(active_elm_indices[0], dtype='int')
        print(f'ELM start indices: {elm_start_indices}')
        elm_stop_indices = np.array(active_elm_indices[-1], dtype='int')
        print(f'ELM stop indices: {elm_stop_indices}')
        valid_t0 = valid_t0_np
        print(f'valid_t0: {valid_t0}')
        signals = signals_np
        labels = labels_np
        print(f'Signals shape: {signals.shape}, labels shape: {labels.shape}')
    else:
        final_idx = labels.shape[0] - 1
        window_start_indices = np.append(window_start_indices, final_idx + 1)
        print(f'window start indices: {window_start_indices}')
        elm_start_indices = np.append(elm_start_indices, final_idx + active_elm_indices[0] + 1)
        print(f'ELM start indices: {elm_start_indices}')
        elm_stop_indices = np.append(elm_stop_indices, final_idx + active_elm_indices[-1] + 1)
        print(f'ELM stop indices: {elm_stop_indices}')
        valid_t0 = np.concatenate([valid_t0, valid_t0_np])
        print(f'valid_t0: {valid_t0}')
        signals = np.concatenate([signals, signals_np], axis=0)
        labels = np.concatenate([labels, labels_np], axis=0)
        print(f'Signals shape: {signals.shape}, labels shape: {labels.shape}')

        

-----Iteration: 0-----
valid_t0_np: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Active ELM indices: [ 7  8  9 10 11]
window start indices: [0]
ELM start indices: 7
ELM stop indices: 11
valid_t0: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Signals shape: (20, 8, 8), labels shape: (20,)
-----Iteration: 1-----
valid_t0_np: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Active ELM indices: [ 9 10 11 12 13 14]
window start indices: [ 0 20]
ELM start indices: [ 7 29]
ELM stop indices: [11 34]
valid_t0: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
Signals shape: (44, 8, 8), labels shape: (44,)
-----Iteration: 2-----
valid_t0_np: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0.]
Active ELM indices: [ 5  6  7  8  9 10 11 12]
window start indices: [ 0 20 44]
ELM start indices: [ 7 29 49]
ELM stop indices: [11 34 56

In [5]:
labels = np.array([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0])
elm_start = 7
elm_stop = 11
fraction_elm = np.count_nonzero(labels >= 0.5) / labels.shape[0]
oversample_count = int((1-fraction_elm)/fraction_elm) - 1
print(f"  Active ELM fraction (raw data): {fraction_elm:.3f}")
print(f"  Active ELM oversampling for balanced data:{oversample_count}")


  Active ELM fraction (raw data): 0.250
  Active ELM oversampling for balanced data:2


In [6]:
active_elm_window = np.arange(elm_start - 3, elm_stop + 3, dtype='int')
active_elm_window

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])

In [7]:
active_elm_window = np.tile(active_elm_window, [oversample_count])
active_elm_window

array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  4,  5,  6,  7,  8,  9, 10,
       11, 12, 13])

In [19]:
valid_t0 = np.ones(labels.shape, dtype='int')
signal_window = 4
valid_t0[-signal_window+1:] = 0
valid_t0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0])

In [20]:
valid_indices = np.arange(valid_t0.size, dtype='int')
valid_indices 

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19])

In [21]:
valid_indices = valid_indices[np.where(valid_t0 > 0)[0]]
sample_indices = valid_indices
sample_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16])

In [22]:
sample_indices = np.concatenate([sample_indices, active_elm_window])
sample_indices

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        4,  5,  6,  7,  8,  9, 10, 11, 12, 13,  4,  5,  6,  7,  8,  9, 10,
       11, 12, 13])

In [23]:
fraction_elm = (
            np.count_nonzero(labels[sample_indices] >= 0.5)
            / sample_indices.size
        )
print(f"  Active ELM fraction (balanced data): {fraction_elm:.3f}")

  Active ELM fraction (balanced data): 0.405
